# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4940, uptime 0:00:34
mini_beamline                    RUNNING   pid 4941, uptime 0:00:34
random_walk                      RUNNING   pid 4942, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4943, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4944, uptime 0:00:34
simple                           RUNNING   pid 4945, uptime 0:00:34
thermo_sim                       RUNNING   pid 4946, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4947, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-05 14:42:35
Persistent Unique Scan ID: '39f32aa8-bbed-4740-b8ff-b3a3026b7377'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 14:42:35.3 |     -1.000 |     100789 |
|         2 | 14:42:35.3 |     -0.500 |     101895 |
|         3 | 14:42:35.4 |      0.000 |     104011 |
|         4 | 14:42:35.4 |      0.500 |     102998 |


|         5 | 14:42:35.4 |      1.000 |     101585 |


+-----------+------------+------------+------------+
generator scan ['39f32aa8'] (scan num: 3)





('39f32aa8-bbed-4740-b8ff-b3a3026b7377',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/39f32aa8-bbed-4740-b8ff-b3a3026b7377-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-05 14:42:35
Persistent Unique Scan ID: '8d776f36-e9e4-4cf1-ab04-40ae5cc7bd89'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 14:42:35.9 |          -1 | 1131335137 |


|         2 | 14:42:36.1 |          -0 | 1131864731 |


|         3 | 14:42:36.2 |           0 | 1130278467 |


|         4 | 14:42:36.3 |           0 | 1127032379 |


|         5 | 14:42:36.4 |           1 | 1122606417 |


+-----------+------------+-------------+------------+
generator scan ['8d776f36'] (scan num: 4)





('8d776f36-e9e4-4cf1-ab04-40ae5cc7bd89',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-05 14:42:36
Persistent Unique Scan ID: '39c464c0-6d2a-4dfc-bdb9-7ec1fd733a70'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 14:42:36.9 | 1101999566 |


|         2 | 14:42:37.8 | 1053895773 |


|         3 | 14:42:38.8 | 1085785826 |


...

|        28 | 14:43:03.9 | 1131618186 |


|        29 | 14:43:04.9 | 1096031958 |


|        30 | 14:43:05.9 | 1053407583 |


+-----------+------------+------------+
generator count ['39c464c0'] (scan num: 5)





('39c464c0-6d2a-4dfc-bdb9-7ec1fd733a70',)